# Navigating Models from FuriosaAI Model Zoo

## FuriosaAI's Software Stack

FuriosaAI's software stack caters to a diverse range of deep learning models, with a primary focus on vision-related tasks. Within this stack, the FuriosaAI Compiler optimizes Deep Neural Network (DNN) models and generates executable code for the FuriosaAI NPU. It currently supports TFLite and ONNX models, utilizing the latest research and methods for optimization. The compiler efficiently accelerates various vision-related operators on the NPU while utilizing the CPU for unsupported operations.

## Vision Models and Beyond

FuriosaAI's first-generation NPU, Warboy, is specialized for vision-related tasks. It accelerates popular vision models like ResNet50, SSD-MobileNet, and EfficientNet, while also enabling users to create custom models that utilize supported operators. This flexibility ensures the generation of highly optimized NPU-ready code for various vision tasks.

## Exploring Vision Models

For easy exploration of vision models tailored for FuriosaAI's NPU, navigate to the `furiosa.models.vision` module. Here, you'll find a curated selection of models that have been optimized for efficient deployment on the FuriosaAI Warboy NPU.


In [1]:
from furiosa.models import vision


# List of available vision models
print(dir(vision))

['EfficientNetB0', 'EfficientNetV2s', 'ResNet50', 'SSDMobileNet', 'SSDResNet34', 'YOLOv5l', 'YOLOv5m']


In [ ]:
# Alternatively, use the Command line tool to list models
! furiosa-models list

|   Model name    |      Model description       |      Task type       | Available postprocesses |
|-----------------|------------------------------|----------------------|-------------------------|
|    ResNet50     |   MLCommons ResNet50 model   | Image Classification |         Python          |
|  SSDMobileNet   | MLCommons MobileNet v1 model |   Object Detection   |      Python, Rust       |
|   SSDResNet34   | MLCommons SSD ResNet34 model |   Object Detection   |      Python, Rust       |
|     YOLOv5l     |      YOLOv5 Large model      |   Object Detection   |          Rust           |
|     YOLOv5m     |     YOLOv5 Medium model      |   Object Detection   |          Rust           |
| EfficientNetB0  |    EfficientNet B0 model     | Image Classification |         Python          |
| EfficientNetV2s |    EfficientNetV2-s model    | Image Classification |         Python          |

Now, let's instantiate a Model class from vision models and delve deeper into its attributes.

In [3]:
model = vision.ResNet50()
print(model)

# Display the static fields of the model
print("Static fields:", list(model.model_fields.keys()))

# Show the lazy-loaded fields of the model
print("Lazy loaded fields:", list(model.model_computed_fields.keys()))

name='ResNet50' task_type=<ModelTaskType.IMAGE_CLASSIFICATION: 'IMAGE_CLASSIFICATION'> format=<Format.ONNX: 'ONNX'> family='ResNet' version='v1.5' metadata=Metadata(description='ResNet50 v1.5 int8 ImageNet-1K', publication=Publication(authors=None, title=None, publisher=None, date=None, url='https://arxiv.org/abs/1512.03385.pdf')) tags=None
Static fields: ['name', 'task_type', 'format', 'family', 'version', 'metadata', 'tags', 'preprocessor', 'postprocessor']
Lazy loaded fields: ['origin', 'tensor_name_to_range']


In [4]:
# Moreover, you can access informative static fields using the Command line tool:
! furiosa-models desc ResNet50

libfuriosa_hal.so --- v0.11.0, built @ 43c901f
name: ResNet50
format: ONNX
family: ResNet
version: v1.5
metadata:
  description: ResNet50 v1.5 int8 ImageNet-1K
  publication:
    url: https://arxiv.org/abs/1512.03385.pdf
task type: Image Classification
available postprocess versions: Python


## Acquire the ENF Binary with `model_source()`

FuriosaAI's Model object offers a method called `model_source()` which allows you to obtain the ENF (FuriosaAI Compiler-specific format) binary for a specific model. This ENF binary can be directly used for further processing or deployment without the need for recompilation. This is particularly beneficial when you want to save time and resources associated with the compilation process.

Using `model_source()` is straightforward. You call this method on a Model object and, as a result, you receive the ENF binary. The `num_pe` parameter, which has a default value of 2, specifies the number of processing elements (PE) to use. You can set it to 1 if you want to use a single PE for the model. This flexibility allows you to optimize the model's deployment according to your specific requirements, whether it's for single-PE or fusioned-PE scenarios.

Here's an example of how to use `model_source()`:

In [5]:
from furiosa.runtime.sync import create_runner

model_source = model.model_source(num_pe=2)

# Create a runner with the model source
with create_runner(model_source) as runner:
    # Print model inputs metadata
    print(runner.model.inputs())
    # Run inferences, ...
    ...

libfuriosa_hal.so --- v0.11.0, built @ 43c901f


2023-10-27T07:37:13.530567Z  WARN furiosa_compiler::api: furiosa-compiler and the dependent IR may be not compatible: (0.10.0#f8f05c8ea != 0.10.1#8b00177dc)
2023-10-27T07:37:13.549259Z  INFO furiosa_rt_core::driver::event_driven::coord: FuriosaRT (v0.10.2, rev: a45bb1a0b, built at: 2023-10-12T06:41:21Z) bootstrapping ...
2023-10-27T07:37:13.553584Z  INFO furiosa_rt_core::driver::event_driven::coord: Found furiosa-compiler (v0.10.0, rev: f8f05c8, built at: 2023-08-08T11:58:09Z)
2023-10-27T07:37:13.553612Z  INFO furiosa_rt_core::driver::event_driven::coord: Found libhal (type: warboy, v0.11.0, rev: 43c901f built at: 2023-08-08T12:07:35Z)
2023-10-27T07:37:13.553622Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-0] detected 1 NPU device(s):
2023-10-27T07:37:13.562958Z  INFO furiosa_rt_core::driver::event_driven::coord: - [0] npu:5:0-1 (warboy-b0-2pe, 128dpes, firmware: 1.7.3276053508, f7b0f28)
2023-10-27T07:37:13.563294Z  INFO furiosa_rt_core::driver::event_driven::coord: [R

:-) Finished in 0.000006756s


2023-10-27T07:37:20.186572Z  INFO furiosa_rt_core::driver::event_driven::coord: [Sess-cf0fc5ce] the model compile is successful (took 3 secs)
2023-10-27T07:37:20.553656Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-0] created 1 NPU threads on npu:5:0-1 (DRAM: 180.0 kiB/16.0 GiB, SRAM: 31.4 MiB/128.0 MiB)
[TensorDesc(shape=(1, 3, 224, 224), dtype=UINT8, format=NCHW, size=150528, len=150528)]
2023-10-27T07:37:20.679167Z  INFO furiosa_rt_core::driver::event_driven::coord: [Sess-cf0fc5ce] terminated
2023-10-27T07:37:20.684148Z  INFO furiosa_rt_core::npu::raw: NPU (npu:5:0-1) has been closed
2023-10-27T07:37:20.689574Z  INFO furiosa_rt_core::driver::event_driven::coord: [Runtime-0] stopped
